In [1]:
import torch
from lightning import Trainer
from dataloader import InsectDatamodule
from model_20 import ResNet
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
# initialize the datamodule

# csv_paths = ['../data/Cicadidae.csv', '../data/Orthoptera.csv']
csv_paths = ['../data/Orthoptera.csv']

batch_size = 10
num_workers = 0

n_fft = 1000
hop_length =  147
win_length = int(hop_length * 2)
n_mels = 64

patience = 30

datamodule = InsectDatamodule(
    csv_paths = csv_paths,
    batch_size = batch_size,
    num_workers = num_workers,
    hop_length = hop_length,
    n_fft = n_fft,
    win_length = win_length,
    n_mels = n_mels)

resnet = ResNet(
    in_channels=1,
    base_channels=8,
    kernel_size=3,
    n_max_pool=3,
    n_res_blocks=4,
    num_classes=datamodule.num_classes,
    learning_rate=0.001,
    class_weights=datamodule.class_weights)

trainer = Trainer(
    log_every_n_steps=20,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=patience),
        ModelCheckpoint(monitor='val_loss', save_top_k=1, mode='min', filename='best'),
    ]
)

c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (64) may be set too high. Or, the value for `n_freqs` (501) may be set too low.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [3]:
trainer.fit(
    resnet,
    train_dataloaders=datamodule.train_dataloader(),
    val_dataloaders=datamodule.val_dataloader()
)

c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\loops\utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type              | Params
------------------------------------------------------------
0 | conv1                 | Conv2d            | 80    
1 | batchnorm1            | BatchNorm2d       | 16    
2 | relu                  | ReLU              | 0     
3 | res_blocks            | Sequential        | 306 K 
4 | avgpool               | AdaptiveAvgPool2d | 0     
5 | convout               | Conv2d            | 1.2 K 
6 | softmax               | Softmax           | 0     
7 | cross_entropy_loss_fn | CrossEntropyLoss  | 0     
------------------------------------------------------------
307 K     Trainable params
0         Non-trainable params
307 K     Total params
1.230     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=20). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2:  80%|████████  | 8/10 [00:01<00:00,  4.41it/s, v_num=9] 

c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [8]:
trainer.test(ckpt_path='best', dataloaders=datamodule.test_dataloader())

Restoring states from the checkpoint path at c:\Users\kraft\OneDrive - ZHAW\MyFiles\001_Studium\001 Semester\04 FS24\07_SemesterArbeitI\SA1_JK\code\lightning_logs\version_8\checkpoints\best.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at c:\Users\kraft\OneDrive - ZHAW\MyFiles\001_Studium\001 Semester\04 FS24\07_SemesterArbeitI\SA1_JK\code\lightning_logs\version_8\checkpoints\best.ckpt
c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 3/3 [00:12<00:00,  0.24it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.5185185074806213
        test_loss           0.19090679287910461
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.19090679287910461, 'test_acc': 0.5185185074806213}]